In [1]:
#AE Project

import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import math as m
import numpy.linalg as la
import scipy.linalg as sla
import csv

#Boeing Insitu ScanEagle Numbers (ON EARTH)
g = 9.81 #m/s^2
w1 = (18+14)/2 #avg empty structure weight in kg
w0 = 22 #kg (max takeoff weight)
Vmax = 41.2 #m/s max horisontal speed
Vcruise = (30 + 25)/2 #avg cruise speed in m/s
b = 3.11 #m (wingspan)
c = 3.11/13 #m (chord length using pixel counting)
Ceiling = 5944 #m
ServiceC = 4572 #m
Sref = b*c

#Boeing Insitu ScanEagle Numbers (ON TITAN)
gt = 1.352 #m/s^2 (t = Titan)
w1t = (w1)*gt #avg empty structure weight in kg
w0t = (w0)*gt #max takeoff weight in kg

##calculating angle of attack via cl/cd for steady level flight
def clcd(rho,Pr):
    Sref = b*c
    cl = w0t/(.5*rho*Sref*Vcruise**2)
    Tr = Pr/Vcruise
    cd = cl/(w0t/Tr)
    ratio = cl/cd #naca files use cl/cd vs alpha to determine angle of attack
    return cl,cd,ratio


##Takeoff
#cd0,cl,e depend on wings
#Tr depends on engine
#muR depends on where it lands in regard to ice or rock
def Takeoff_Landing(cd0,cl,e,muR,rho): 
    Sref = b*c
    Tr = 1.2/Vcruise
    H = (60-9)/(365/1.55) #m height of wing above ground w/o wheels (will change if airfoil found, done using pixel math)
    phi = ((16*(H/b))**2)/(1+((16*(H/b))**2)) #ground effect m
    AR = (b**2)/Sref
    CD = cd0 + (phi*(cl**2)/(m.pi*e*AR))
    Vav = 0.7*1.2*m.sqrt((2*w0t)/(rho*Sref*cl))
    L = .5*rho*(Vav**2)*Sref*cl
    D = .5*rho*(Vav**2)*Sref*CD
    Slo = (1.44*w0t**2)/(gt*rho*Sref*cl*(Tr - (D+muR*(w0t-L)))) #takeoff
    Sl = (1.69*w0t**2)/(gt*rho*Sref*cl*(D+muR*(w0t-L)))
    return Slo,Sl

def rc(n,cd0,W,cL,rho):
    Sref = b*c
    V = m.sqrt((2*w)/(rho*Sref*cL))
    CD = cd0 + (phi*(cl**2)/(m.pi*e*AR))
    L = .5*rho*(V**2)*Sref*cl
    D = .5*rho*(V**2)*Sref*CD
    Tr = W/(L/D)
    AR = (b**2)/Sref
    CD = cd0 + (phi*(cl**2)/(m.pi*e*AR))
    Pr = Tr*V
    Pa = n*P
    rc = (Pa - Pr)/2
    return rc


In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

#read csv
temp = []
altitudes = []
with open("TitanTempAlt.csv",'r') as csvfile:
    plots = csv.reader(csvfile, dialect='excel', delimiter=',')
    for row in plots:
        try:
            altitudes.append(float(row[0]))
            temp.append(float(row[1]))
        except:
            continue
            

#helper function that linearly interpolates a value at a given altitude between two altitudes
def linear_interpolator(para1, para2, alt1, alt2, alt):
    alt_ratio = (alt - alt1) / (alt2 - alt1)
    return ((para2 - para1) * alt_ratio) + para1

def atms_conditions(alt):
    #get alt in km to work with csv data which is in km
    alt = alt/1000
    if alt > altitudes[-1] or alt <= 0:
        print("sorry this altitude is not in range of our function")
        return
    
    P0 = 146700 #Pa
    R  = 296.9  #J/kg * K
    rho0 = P0 / (temp[0] * R)
    g = 1.352 #m/s^2; just googled value for titan
    #exponential relation for density
    #print(alt)
    R_titan = 2575 #km
    P1 = P0 * math.exp(-alt/(R_titan))
    #print(rho0)
    iterator = 0 #describes state state of while loop (ie what altitude and temp to be at)
    next_temp = temp[0]
    #lists akin to vectors, allow for dynamic storage of values
    while altitudes[iterator] < alt:
        #get the temp and altitude of the next state of function
        next_temp = temp[iterator + 1]
        next_altitude = altitudes[iterator + 1]
        iterator += 1
    
    T_ceil = temp[iterator]
    T_floor = temp[iterator - 1]
    alt_ceil = altitudes[iterator]
    alt_floor = altitudes[iterator - 1]
    T1 = linear_interpolator(T_floor, T_ceil, alt_floor, alt_ceil, alt)
    #print(T_floor, T_ceil, T1)
    
    rho1 = P1 / (R * T1)
    
    return alt*1000, P1, T1, rho1

print(atms_conditions(5000))
print(atms_conditions(100000))

In [3]:
#calculating Reynolds number
mu = 1.66*10**-5 #kg/m*s viscosity for Nitrogen gas
R = (atms_conditions(5)[3]*Vcruise*c)/mu
print("Reynolds number =",R)
print("cl,cd, cl/cd =", clcd(atms_conditions(5)[3],(1.2)))

def e(cd,cl):
    cd0 = -0.0087272727 #cd at 0 angle of attack?
    Sref = b*c
    AR = (b**2)/Sref
    e = (cl**2)/(m.pi*AR*(cd-cd0))
    return e

print("oswald e =", e(clcd(atms_conditions(5)[3],(1.2))[1],clcd(atms_conditions(5)[3],(1.2))[0]))

print("takeoff, landing =", Takeoff_Landing(-0.0087272727,clcd(atms_conditions(5)[3],(1.2))[0],e(clcd(atms_conditions(5)[3],(1.2))[1],clcd(atms_conditions(5)[3],(1.2))[0]),((.75+.55)/2),atms_conditions(5)[3])) #generic rock muR


Reynolds number = 2090167.77037195
cl,cd, cl/cd = (0.020046891987303646, 2.9410081647958788e-05, 681.6333333333334)
oswald e = 0.0011237281453171006
takeoff, landing = (-7538.588164274349, 8610.905843852044)


In [4]:
num = 1.44*w0t**2
denom = gt*4*(3.11/13)*3.11*1.2
print(denom)
coef = 0.09*(w0t-(.5*4*(28.3**2)*(3.11/13)*3.11*0.013))
print(coef)
T = (num/(denom*(10-m.sqrt(30))))+coef
print(T)

4.828312319999999
1.2826281295800004
59.62189330005131


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import math as m
import numpy.linalg as la
import scipy.linalg as sla
import csv

def T_a(cl):
    contraeff = (0.06+0.16)/2
    vlo = 1.2*m.sqrt((2*w0t)/(atms_conditions(5000)[3]*Sref*cl))
    T_a = (0.8*contraeff*1.2*(10**3))/(vlo)
    return cl

    
mu = 1.66*10**-5 #kg/m*s viscosity for Nitrogen gas
R = (atms_conditions(5000)[3]*25*c)/mu
print("Reynolds number =",R)

Reynolds number = 2123689.4390487773
